In [99]:
# 필요한 라이브러리들을 로드하자
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import undetected_chromedriver as uc

In [100]:
# 크롬 웹드라이버를 실행해서 로그인하는 과정을 함수로 나타낸거

def init_driver():
    driver = uc.Chrome()
    driver.get('https://velog.io')
    return driver
# 로그인 버튼을 눌러주고 로그인이 되서 다시 velog로 돌아올때까지 60초를 기다린다.
def do_login(driver):
    driver.find_element(By.XPATH,'//button[text()="로그인"]').click()
    WebDriverWait(driver, 60).until(
        EC.presence_of_element_located(
            (By.XPATH, '//button[text()="새 글 작성"]')
        )
    )

In [101]:
if  __name__  ==  "__main__" :
    driver = init_driver()
    do_login(driver)

In [102]:
# G10 ESI 데이터가 있는 곳을 웹드라이버로 실행시켜보자
url = 'https://en.macromicro.me/charts/9700/global-trade-sk'  # 원하는 웹사이트 주소로 변경하세요
driver.get(url)

In [103]:
# 로그인 창이 뜰거임. 로그인 하기위한 첫 번째 과정
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//a[@href="/login?next=%2Fcharts%2F45866%2Fglobal-citi-surprise-index"]'))
)

# 찾은 요소를 클릭합니다.
element.click()

KeyboardInterrupt: 

In [ ]:
# 로그인 하기위한 두 번째 과정
element2 = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//a[@href="/login/google"]'))
)

# 찾은 요소를 클릭합니다.
element2.click()

In [104]:
element = driver.find_element(By.CSS_SELECTOR, f'.highcharts-legend-item.highcharts-line-series.highcharts-color-undefined.highcharts-series-1')
element.click()

element = driver.find_element(By.CSS_SELECTOR, f'.highcharts-legend-item.highcharts-column-series.highcharts-color-undefined.highcharts-series-2')
element.click()

In [105]:
# 마우스를 움직이고 클릭하기 위한 액션을 정의해줌
actions = ActionChains(driver)

In [106]:
# 내가 수동으로 찾은 초기 커서 위치로서 적합한 커서 위치
element3 = driver.find_element(By.CSS_SELECTOR, '.highcharts-axis.highcharts-yaxis')

# 찾은 요소를 클릭합니다.
actions.click(element3).perform()   # 우클릭 하고 싶으면 actions.context_click(element3).perform()
# actions.context_click(element3).perform()

In [107]:
# 첫 데이터가 나타나는 곳으로 커서를 이동후 클릭(내가 수동으로 찾음)
actions.move_by_offset(40, 0).click().perform()

In [108]:
# 커서를 하나씩 이동 시키면서 커서를 갖다 댈때만 뜨는 데이터 값들을 크롤링 해보자

data_list = []

for i in range(910):

    # 커서를 한 칸씩 옮겨보자
    actions.move_by_offset(1, 0).click().perform()

    # 페이지의 HTML을 가져옴
    html = driver.page_source
    # BeautifulSoup 객체를 생성
    soup = bs(html, 'html.parser')
    # 클래스가 "chart-wrapper"인 요소를 찾아서 해당하는 것을 data에 저장
    wrapper = soup.find('g', attrs={'class': 'highcharts-label highcharts-tooltip highcharts-color-undefined'})
    data = wrapper.prettify()

    # 데이터에서 요일, 월, 일, 연도 추출
    date_start_index = data.find("<tspan style=\"font-size: 10px;\">") + len("<tspan style=\"font-size: 10px;\">")
    date_end_index = data.find("</tspan>")
    date = data[date_start_index:date_end_index].strip()
    # 날짜 잘 나오나 함 찍어보자
    print(date)

    # 데이터에서 G10 ESI 값을 추출
    number_start_index = data.find("<tspan style=\"font-weight: bold;\">") + len("<tspan style=\"font-weight: bold;\">")
    number_end_index = data.find("</tspan>", number_start_index)
    number = data[number_start_index:number_end_index].strip()
    # ESI값 잘 나오나 함 찍어보자
    print(number)

    # 추출한 정보를 리스트에 형태로 하나씩 추가
    data_list.append({'date': date, 'number': number})

# 커서를 하나씩 이동 시켜서 하나씩 크롤링 하는 작업을 끝냈으면 해당 데이터를 데이터 프레임으로 저장
df = pd.DataFrame(data_list)

February 2000
35.80
February 2000
35.80
February 2000
35.80
March 2000
23.80
March 2000
23.80
March 2000
23.80
April 2000
17.60
April 2000
17.60
April 2000
17.60
May 2000
28.10
May 2000
28.10
May 2000
28.10
May 2000
28.10
June 2000
19.00
June 2000
19.00
June 2000
19.00
July 2000
23.00
July 2000
23.00
July 2000
23.00
August 2000
30.10
August 2000
30.10
August 2000
30.10
September 2000
26.50
September 2000
26.50
September 2000
26.50
September 2000
26.50
October 2000
13.40
October 2000
13.40
October 2000
13.40
November 2000
5.60
November 2000
5.60
November 2000
5.60
December 2000
0.10
December 2000
0.10
December 2000
0.10
January 2001
4.00
January 2001
4.00
January 2001
4.00
January 2001
4.00
February 2001
5.30
February 2001
5.30
February 2001
5.30
March 2001
-2.10
March 2001
-2.10
March 2001
-2.10
April 2001
-10.40
April 2001
-10.40
April 2001
-10.40
May 2001
-9.10
May 2001
-9.10
May 2001
-9.10
May 2001
-9.10
June 2001
-15.20
June 2001
-15.20
June 2001
-15.20
July 2001
-21.20
July 2001
-

In [109]:
# 생성한 데이터 프레임에서 중복되는 값은 삭제
df_droped = df.drop_duplicates(subset='date')

# 그리고 중복되는 데이터를 제거한 완성된 데이터를 csv의 파일형태로 저장해둔다
df_droped.to_csv('export.csv', index=False)

In [112]:
pd.read_csv('export.csv')

,date,number
0,February 2000,35.8
1,March 2000,23.8
2,April 2000,17.6
3,May 2000,28.1
4,June 2000,19.0
...,...,...
275,January 2023,-16.4
276,February 2023,-7.6
277,March 2023,-13.8
278,April 2023,-14.3
